## 1. Ambiente y Craga de Datos


In [ ]:
import pandas as pd
from google.colab import drive

drive.mount('/content/drive')

#Carga el archivo retail_sales.csv en un DataFrame de Pandas.
path = "/content/drive/MyDrive/SONDA | Data Science/Data Science Fundamentals/BBDD/retail_sales_dataset.csv"
df = pd.read_csv(path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 2. Transformación de Datos
Crea nuevas columnas: Basándonos en los datos existentes, crea nuevas columnas que sean útiles para el análisis. Por ejemplo, calcula el ingreso total por venta y normaliza las ventas.

In [ ]:
df.head()


,Transaction ID,Date,Customer ID,Gender,Age,Product Category,Quantity,Price per Unit,Total Amount
0,1,2023-11-24,CUST001,Male,34,Beauty,3,50,150
1,2,2023-02-27,CUST002,Female,26,Clothing,2,500,1000
2,3,2023-01-13,CUST003,Male,50,Electronics,1,30,30
3,4,2023-05-21,CUST004,Male,37,Clothing,1,500,500
4,5,2023-05-06,CUST005,Male,30,Beauty,2,50,100


In [ ]:
#, calcula el ingreso total por venta y normaliza las ventas.

min_val = df["Total Amount"].min()
max_val = df["Total Amount"].max()

df["Ventas Normalizadas"] = (df["Total Amount"] - min_val) / (max_val - min_val)


Clasifica los datos: Crea una columna que clasifique las ventas en categorías significativas (e.g., ‘Alta’, ‘Media’, ‘Baja’).

In [ ]:
#se crea funcion que recibe como param de entrada un decimal (que correspondera a la venta normalizada)
def categ_ventas(x: float):
  if x > 0.5 : return "Alta"
  elif  x  < 0.5: return "Baja"
  elif x == 0.5: return "Media"

df["Categoria Venta"] = df["Ventas Normalizadas"].apply(lambda x: categ_ventas(x))
df.tail()

,Transaction ID,Date,Customer ID,Gender,Age,Product Category,Quantity,Price per Unit,Total Amount,Ventas Normalizadas,Categoria Venta
995,996,2023-05-16,CUST996,Male,62,Clothing,1,50,50,0.012658,Baja
996,997,2023-11-17,CUST997,Male,52,Beauty,3,30,90,0.032911,Baja
997,998,2023-10-29,CUST998,Female,23,Beauty,4,25,100,0.037975,Baja
998,999,2023-12-05,CUST999,Female,36,Electronics,3,50,150,0.063291,Baja
999,1000,2023-04-12,CUST1000,Male,47,Electronics,4,30,120,0.048101,Baja


## Agrupación y Agregación
Agrupación por múltiples columnas: Realiza agrupaciones por categorías como producto y tienda, producto y mes, etc.


In [ ]:
#no existen las columnas tienda ni producto en el data set, utilizare otras para desarrollar este item
df.groupby("Gender")["Total Amount"].sum()
df.groupby("Product Category")["Total Amount"].sum()
df.groupby(["Gender", "Product Category"])["Total Amount"].sum()

#transformo a datetime la columna Date para obtener mes (esta limpieza y transformacion se realizcó en parte II)
df["Date"] = pd.to_datetime(df["Date"])
df.groupby([df["Date"].dt.year, df["Date"].dt.month_name()])["Total Amount"].sum()

Date  Date     
2023  April        33870
      August       36960
      December     44690
      February     44060
      January      35450
      July         35465
      June         36715
      March        28990
      May          53150
      November     34920
      October      46580
      September    23620
2024  January       1530
Name: Total Amount, dtype: int64

Aplicar funciones de agregación: Utiliza funciones como sum, mean, count, min, max, std, y var para obtener estadísticas descriptivas de cada grupo.

In [ ]:
df["Total Amount"].mean() #media
df["Total Amount"].std() #desviacion estandar
df["Total Amount"].min()# venta minima
df["Total Amount"].max() # venta mas alta
df["Total Amount"].describe() #ver estadisticas descriptivas

,Total Amount
count,1000.000000
mean,456.000000
std,559.997632
min,25.000000
25%,60.000000
50%,135.000000
75%,900.000000
max,2000.000000


## Análisis Personalizado con apply
Función personalizada: Aplica funciones personalizadas para realizar análisis específicos que no se pueden lograr con las funciones de agregación estándar.
Ejemplo de uso avanzado: Calcula la desviación de cada venta respecto a la media de su grupo.


In [ ]:
#creamos la funcion que recibe como parametro de entrada una fila
def mean_group(row):
    categoria = row["Product Category"]
    venta = row["Total Amount"]
    media = df[df["Product Category"] == categoria]["Total Amount"].mean() #calculando media de la categoria
    return venta - media #esviación de cada venta respecto a la media de su grupo.

df.apply(mean_group, axis=1) #axis 1 porque recibe mas de una columna




,0
0,-317.475570
1,556.752137
2,-428.786550
3,56.752137
4,-367.475570
...,...
995,-393.247863
996,-377.475570
997,-367.475570
998,-308.786550


In [ ]:
def cat_edad(edad):
    if edad < 18:
        return "Menor"
    elif edad < 40:
        return "Joven"
    elif edad < 65:
        return "Adulto"
    else:
        return "Mayor"

df["Categoria_Edad"] = df["Age"].apply(lambda x: cat_edad(x))
df.head()

,Transaction ID,Date,Customer ID,Gender,Age,Product Category,Quantity,Price per Unit,Total Amount,Ventas Normalizadas,Categoria Venta,Desviacion_vs_Media,Categoria_Edad
0,1,2023-11-24,CUST001,Male,34,Beauty,3,50,150,0.063291,Baja,-317.475570,Joven
1,2,2023-02-27,CUST002,Female,26,Clothing,2,500,1000,0.493671,Baja,556.752137,Joven
2,3,2023-01-13,CUST003,Male,50,Electronics,1,30,30,0.002532,Baja,-428.786550,Adulto
3,4,2023-05-21,CUST004,Male,37,Clothing,1,500,500,0.240506,Baja,56.752137,Joven
4,5,2023-05-06,CUST005,Male,30,Beauty,2,50,100,0.037975,Baja,-367.475570,Joven
